In [1]:
from keras.layers import *
from keras.models import Model
from keras import layers
from keras.layers.merge import concatenate
import sys
sys.path.insert(1, '../src')
sys.path.insert(1, '../image_segmentation_keras')
from keras_segmentation.models.config import IMAGE_ORDERING

from keras_segmentation.models.model_utils import get_segmentation_model
from glob import glob
from crfrnn_layer import CrfRnnLayer

Using TensorFlow backend.


Error in loading augmentation, can't import imgaug.Please make sure it is installed.


In [2]:
input_height = 256
input_width = 256
n_classes = 3
channels = 3

In [3]:
def unet_conv_block(inputs, filters, pool=True, batch_norm_first=True):
    if batch_norm_first == True:
        x = Conv2D(filters, 3, padding="same")(inputs)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

        x = Conv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
    elif batch_norm_first == False:
        x = Conv2D(filters, 3, padding="same")(inputs)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)

        x = Conv2D(filters, 3, padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization()(x)

    if pool == True:
        p = MaxPooling2D((2, 2))(x)
        return [x, p]
    else:
        return x

In [ ]:
img_input = Input(shape=(input_height,input_width, channels))
f1 = unet_conv_block(img_input, 64, pool=True, batch_norm_first=True)
pool1 = MaxPooling2D((2, 2))(f1)
f2 = unet_conv_block(pool1, 128, pool=True, batch_norm_first=True)
pool2 = MaxPooling2D((2, 2))(f2)
f3 = unet_conv_block(pool2, 256, pool=True, batch_norm_first=True)
pool3 = MaxPooling2D((2, 2))(f3)
f4 = unet_conv_block(pool3, 512, pool=True, batch_norm_first=True)
pool4 = MaxPooling2D((2, 2))(f4)
f5 = unet_conv_block(f4[1], 1024, pool=False, batch_norm_first=True)

 
x = BatchNormalization()(x)
x = Activation('relu')(x)
crf_output = CrfRnnLayer(image_dims=(input_height, input_width),
                         num_classes=n_classes,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([x, img_input])
model = get_segmentation_model(img_input, crf_output)

In [4]:
img_input = Input(shape=(input_height,input_width, channels))
f1 = unet_conv_block(img_input, 64, pool=True, batch_norm_first=False)
f2 = unet_conv_block(f1[1], 128, pool=True, batch_norm_first=False)
f3 = unet_conv_block(f2[1], 256, pool=True, batch_norm_first=False)
f4 = unet_conv_block(f3[1], 512, pool=True, batch_norm_first=False)
f5 = unet_conv_block(f4[1], 1024, pool=False, batch_norm_first=False)

x = UpSampling2D((2, 2))(f5)
x = concatenate([x, f4[0]], axis=3)
x = unet_conv_block(x, 512, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f3[0]], axis=3)
x = unet_conv_block(x, 256, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f2[0]], axis=3)
x = unet_conv_block(x, 128, pool=False, batch_norm_first=True)

x = UpSampling2D((2, 2))(x)
x = concatenate([x, f1[0]], axis=3)
x = unet_conv_block(x, 64, pool=False, batch_norm_first=True)

x = Conv2D(n_classes, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
crf_output = CrfRnnLayer(image_dims=(input_height, input_width),
                         num_classes=n_classes,
                         theta_alpha=160.,
                         theta_beta=3.,
                         theta_gamma=3.,
                         num_iterations=10,
                         name='crfrnn')([x, img_input])
model = get_segmentation_model(img_input, crf_output)


In [5]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         activation_1[0][0]               
____________________________________________________________________________________________

In [ ]:
model.train(
    train_images =  "/Users/mavaylon/Research/Data1/train/img/",
    train_annotations = "/Users/mavaylon/Research/Data1/train/ann/",
    epochs=20,
    steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/train/img/*")),
    batch_size=1,
    validate=True,
    val_images="/Users/mavaylon/Research/Data1/test/img/",
    val_annotations="/Users/mavaylon/Research/Data1/test/ann/",
    val_batch_size=1,
    val_steps_per_epoch=len(glob("/Users/mavaylon/Research/Data1/test/img/*"))
)

Verifying training dataset


  2%|▏         | 32/1478 [00:00<00:04, 318.33it/s]

Dataset verified! 
Verifying validation dataset


100%|██████████| 1478/1478 [00:04<00:00, 335.44it/s]


Dataset verified! 
fit
Epoch 1/20
5912/5912 [==============================] - 21110s 4s/step - loss: 0.6242 - accuracy: 0.7598 - val_loss: 0.4549 - val_accuracy: 0.7896

Epoch 00001: val_accuracy improved from -inf to 0.78955, saving model to pet_class_crf.h5
Epoch 2/20
5912/5912 [==============================] - 20313s 3s/step - loss: 0.5241 - accuracy: 0.8040 - val_loss: 0.4298 - val_accuracy: 0.7997

Epoch 00002: val_accuracy improved from 0.78955 to 0.79972, saving model to pet_class_crf.h5
Epoch 3/20
5912/5912 [==============================] - 20803s 4s/step - loss: 0.4708 - accuracy: 0.8255 - val_loss: 0.4460 - val_accuracy: 0.8000

Epoch 00003: val_accuracy improved from 0.79972 to 0.80004, saving model to pet_class_crf.h5
Epoch 4/20
5912/5912 [==============================] - 20825s 4s/step - loss: 0.4306 - accuracy: 0.8416 - val_loss: 0.4181 - val_accuracy: 0.8253

Epoch 00004: val_accuracy improved from 0.80004 to 0.82530, saving model to pet_class_crf.h5
Epoch 5/20
5912/